## Set up:

In [ ]:
import os, sys

import warnings

sys.path.append(os.path.join(os.getcwd(), '../../')) # Add root of repo to import MBM

import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import pandas as pd
import re
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point
import contextily as cx
import matplotlib.colors as mcolors
from shapely.geometry import box
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
import massbalancemachine as mbm
from cmcrameri import cm
from collections import defaultdict
from scipy.ndimage import gaussian_filter

from scripts.geodata import *
from scripts.geodata_plots import *
from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.xgb_helpers import *
from scripts.config_CH import *
#  Suppress warnings issued by Cartopy when downloading data files
warnings.filterwarnings('ignore')

# reload modules
%load_ext autoreload
%autoreload 2

# config file
cfg = mbm.SwitzerlandConfig()

cfg.dataPath

In [ ]:
# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)

cmap = cm.devon
color_palette_glaciers = sns.color_palette(get_cmap_hex(cmap, 15))

colors = get_cmap_hex(cm.batlow, 2)
color_xgb = colors[0]
color_tim = '#c51b7d'

In [ ]:
vois_climate = [
    't2m_corr', 'tp_corr', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10'
]
vois_topographical = [
    "aspect_sgi",
    "slope_sgi",
    "hugonnet_dhdt",
    "consensus_ice_thickness",
    "millan_v",
]
# Feature columns:
feature_columns = [
    'ELEVATION_DIFFERENCE'
] + list(vois_climate) + list(vois_topographical) + ['pcsr']
all_columns = feature_columns + cfg.fieldsNotFeatures

In [ ]:
# === Load RGI glacier IDs ===
rgi_df = pd.read_csv(cfg.dataPath + path_glacier_ids)
rgi_df.columns = rgi_df.columns.str.strip()
rgi_df = rgi_df.sort_values(by='short_name').set_index('short_name')

# === Load SGI region geometries ===
SGI_regions = gpd.read_file(
    os.path.join(cfg.dataPath, path_SGI_topo, 'inventory_sgi2016_r2020',
                 'sgi_regions.geojson'))

# Clean object columns
SGI_regions[SGI_regions.select_dtypes(include='object').columns] = \
    SGI_regions.select_dtypes(include='object').apply(lambda col: col.str.strip())

SGI_regions = SGI_regions.drop_duplicates().dropna()
SGI_regions = SGI_regions.set_index('pk_sgi_region')

# === Map to Level 0 river basins ===
catchment_lv0 = {
    'A': 'Rhine',
    'B': 'Rhone',
    'C': 'Po',
    'D': 'Adige',
    'E': 'Danube'
}
rgi_df['rvr_lv0'] = rgi_df['sgi-id'].str[0].map(catchment_lv0)


# === Map to Level 1 river basins using SGI regions ===
def get_river_basin(sgi_id):
    key = sgi_id.split('-')[0]
    if key not in SGI_regions.index:
        return None
    basin = SGI_regions.loc[key, 'river_basin_name']
    if isinstance(basin, pd.Series):
        return basin.dropna().unique()[0] if not basin.dropna().empty else None
    return basin if pd.notna(basin) else None


rgi_df['rvr_lv1'] = rgi_df['sgi-id'].apply(get_river_basin)

# Final formatting
rgi_df = rgi_df.reset_index().rename(columns={
    'short_name': 'GLACIER'
}).set_index('GLACIER')
rgi_df.head()

In [ ]:
PATH_PREDICTIONS = cfg.dataPath + path_distributed_MB_glamos + 'MBM/glamos_dems_corr/'
# PATH_PREDICTIONS = cfg.dataPath + path_distributed_MB_glamos + 'MBM/glamos_dems_NN/'

## Glaciers covered by GLAMOS:

In [ ]:
glDirect = np.sort([
    re.search(r'xr_direct_(.*?)\.zarr', f).group(1)
    for f in os.listdir(cfg.dataPath + path_pcsr + 'zarr/')
])
geodetic_mb = get_geodetic_MB(cfg)

# filter to glaciers with potential clear sky radiation data
geodetic_mb = geodetic_mb[geodetic_mb.glacier_name.isin(glDirect)]

# get years per glacier
years_start_per_gl = geodetic_mb.groupby(
    'glacier_name')['Astart'].unique().apply(list).to_dict()
years_end_per_gl = geodetic_mb.groupby('glacier_name')['Aend'].unique().apply(
    list).to_dict()

periods_per_glacier, geoMB_per_glacier = build_periods_per_glacier(geodetic_mb)

In [ ]:
# List the files in both directories
glaciers_in_distributed_glamos = os.listdir(cfg.dataPath + path_distributed_MB_glamos +
                                            'MBM/glamos_dems/')
glaciers_in_geodetic = periods_per_glacier.keys()
glaciers_in_glamos = os.listdir(PATH_PREDICTIONS)

# Check if all glaciers in the first directory are in the second directory
missing_glaciers = [
    glacier for glacier in glaciers_in_distributed_glamos
    if glacier not in glaciers_in_glamos
]
missing_glaciers_geod = [
    glacier for glacier in glaciers_in_geodetic
    if glacier not in glaciers_in_glamos
]

# Find the intersection of the two lists
intersecting_glaciers = list(
    set(missing_glaciers_geod) & set(missing_glaciers))

print("The following glaciers are missing compared to gridded MB:")
print(missing_glaciers)

print("The following glaciers don't have geodetic MB:")
print(missing_glaciers_geod)

print("The following glaciers are missing in both lists:")
print(intersecting_glaciers)

In [ ]:
# Glaciers with geodetic MB data:
# Sort glaciers by area
gl_area = get_gl_area(cfg)
gl_area['clariden'] = gl_area['claridenL']


# Sort the lists by area if available in gl_area
def sort_by_area(glacier_list, gl_area):
    return sorted(glacier_list, key=lambda g: gl_area.get(g, 0), reverse=False)


glacier_list = [
    f for f in list(periods_per_glacier.keys()) if f in glaciers_in_glamos
]
glacier_list = sort_by_area(glacier_list, gl_area)
# print len and list
print('Number of glaciers:', len(glacier_list))
print('Glaciers:', glacier_list)

## Geodetic MB:

#### Example:

In [ ]:
glacier_name = "silvretta"
GLAMOS_glwmb = get_GLAMOS_glwmb(glacier_name, cfg)

# If GLAMOS data is missing, skip processing
if GLAMOS_glwmb is None:
    print(f"Skipping {glacier_name}: No GLAMOS data available.")
else:
    # Get all geodetic periods
    periods = periods_per_glacier.get(glacier_name, [])
    geoMBs = geoMB_per_glacier.get(glacier_name, [])

    path_mbm_pred = os.path.join(PATH_PREDICTIONS, glacier_name)

    # Storage lists
    mbm_mb_mean, glamos_mb_mean, geodetic_mb, target_period = [], [], [], []

    for period in periods:
        mbm_mb, glamos_mb = [], []

        for year in range(period[0], period[1] + 1):
            # Construct file path
            file_path = os.path.join(cfg.dataPath, path_mbm_pred,
                                     f"{glacier_name}_{year}_annual.zarr")

            # Check if the NetCDF file exists
            if not os.path.exists(file_path):
                print(
                    f"Warning: Missing MBM file for {glacier_name} ({year}). Skipping..."
                )
                mbm_mb.append(np.nan)
            else:
                ds = xr.open_dataset(file_path)
                # Compute mean glacier-wide MB
                mbm_mb.append(ds["pred_masked"].mean().values)

            # Get GLAMOS Balance for the year, or return NaN if missing
            glamos_mb.append(GLAMOS_glwmb["GLAMOS Balance"].get(year, np.nan))

        # Store mean values, ignoring NaNs
        mbm_mb_mean.append(np.nanmean(mbm_mb))
        glamos_mb_mean.append(np.nanmean(glamos_mb))
        geodetic_mb.append(geoMBs[periods.index(period)])
        target_period.append(period)

    # store all in a dataframe
    df = pd.DataFrame({
        'mbm_mb_mean': mbm_mb_mean,
        'glamos_mb_mean': glamos_mb_mean,
        'geodetic_mb': geodetic_mb,
        'target_period': target_period,
        'end_year': [period[1] for period in target_period],
        'start_year': [period[0] for period in target_period],
    })

fig = plot_geodetic_MB(df, glacier_name, color_xgb, color_tim)

#### All glaciers:

In [ ]:
test_glaciers = [
    'tortin', 'plattalva', 'sanktanna', 'schwarzberg', 'hohlaub', 'pizol',
    'corvatsch', 'tsanfleuron', 'forno'
]
df_all = process_geodetic_mass_balance_comparison(
    glacier_list=glacier_list,
    path_SMB_GLAMOS_csv=cfg.dataPath+path_SMB_GLAMOS_csv,
    periods_per_glacier=periods_per_glacier,
    geoMB_per_glacier=geoMB_per_glacier,
    gl_area=gl_area,
    test_glaciers=test_glaciers,
    path_predictions=PATH_PREDICTIONS,  # or another path if needed
    cfg = cfg
)

#### One plot:

In [ ]:
# All glaciers
scatter_geodetic_MB(df_all, hue='GLACIER', size=True)

In [ ]:
size = True

# Drop rows where any required columns are NaN
df_all = df_all.dropna(subset=['Geodetic MB', 'MBM MB', 'GLAMOS MB'])

# Compute RMSE and Pearson correlation
rmse_mbm = mean_squared_error(df_all["Geodetic MB"],
                              df_all["MBM MB"],
                              squared=False)
corr_mbm = np.corrcoef(df_all["Geodetic MB"], df_all["MBM MB"])[0, 1]
rmse_glamos = mean_squared_error(df_all["Geodetic MB"],
                                 df_all["GLAMOS MB"],
                                 squared=False)
corr_glamos = np.corrcoef(df_all["Geodetic MB"], df_all["GLAMOS MB"])[0, 1]

# Define figure and axes
fig, axs = plt.subplots(1, 1, figsize=(10, 8), sharex=True)

# Plot MBM MB vs Geodetic MB
plot_scatter(df_all, 'GLACIER', size, axs, "MBM MB",
             rmse_mbm, corr_mbm)

axs.set_title('Geodetic target vs Mass Balance Machine', fontsize=24)
axs.set_ylabel('Predicted mass balance [m w.e.]', fontsize=18)
axs.set_xlabel('Geodetic mass balance [m w.e.]', fontsize=18)

# Adjust legend outside of plot
handles, labels = axs.get_legend_handles_labels()
fig.legend(handles,
           labels,
           bbox_to_anchor=(1.05, 1),
           loc="upper left",
           borderaxespad=0.,
           ncol=2,
           fontsize=20)

plt.tight_layout()
plt.show()

## Glacier wide MB:

In [ ]:
# Define glacier name
glacier_name = "basodino"

# Load GLAMOS data
GLAMOS_glwmb = get_GLAMOS_glwmb(glacier_name, cfg)

# Define the path to model predictions
path_results = os.path.join(PATH_PREDICTIONS, glacier_name)

# Extract available years from NetCDF filenames
years = sorted([
    int(f.split("_")[1]) for f in os.listdir(path_results)
    if f.endswith("_annual.zarr")
])

# Extract model-predicted mass balance
pred_gl = []
for year in years:
    file_path = os.path.join(path_results,
                             f"{glacier_name}_{year}_annual.zarr")
    if not os.path.exists(file_path):
        print(
            f"Warning: Missing MBM file for {glacier_name} ({year}). Skipping..."
        )
        pred_gl.append(np.nan)
        continue

    ds = xr.open_dataset(file_path)
    pred_gl.append(ds.pred_masked.mean().item())

# Create DataFrame
MBM_glwmb = pd.DataFrame(pred_gl, index=years, columns=["MBM Balance"])

# Merge with GLAMOS data
MBM_glwmb = MBM_glwmb.join(GLAMOS_glwmb)

# Drop NaN values to avoid plotting errors
MBM_glwmb = MBM_glwmb.dropna()

# Plot the data
fig, ax = plt.subplots(figsize=(12, 6))
MBM_glwmb.plot(ax=ax, marker="o", color=[color_xgb, color_tim])
ax.set_title(f"{glacier_name.capitalize()} Glacier", fontsize=24)
ax.set_ylabel("Mass Balance [m w.e.]", fontsize=18)
ax.set_xlabel("Year", fontsize=18)
ax.grid(True, linestyle="--", linewidth=0.5)

# increase font size of legends
ax.legend(fontsize=14)
plt.show()


In [ ]:
# Define output folder
output_folder = "figures/glacierwide/"
emptyfolder(output_folder)  # Clear existing figures

# Process each glacier
for glacier_name in tqdm(glacier_list):
    # Load GLAMOS data
    GLAMOS_glwmb = get_GLAMOS_glwmb(glacier_name, cfg)

    # Check if GLAMOS data is available
    if GLAMOS_glwmb is None:
        print(f"Skipping {glacier_name}: No GLAMOS data available.")
        continue

    # Define the path to model predictions
    glacier_path = os.path.join(PATH_PREDICTIONS, glacier_name)

    # Check if glacier prediction directory exists
    if not os.path.exists(glacier_path):
        print(f"Skipping {glacier_name}: No prediction data available.")
        continue

    # Extract available years from NetCDF filenames
    try:
        years = sorted([
            int(f.split("_")[1]) for f in os.listdir(glacier_path)
            if f.endswith("_annual.zarr") and "_" in f
        ])
    except ValueError:
        print(
            f"Skipping {glacier_name}: Failed to extract years from filenames."
        )
        continue

    if not years:
        print(f"Skipping {glacier_name}: No model prediction files found.")
        continue

    # Extract model-predicted mass balance
    pred_gl = []
    for year in years:
        file_path = os.path.join(glacier_path,
                                 f"{glacier_name}_{year}_annual.zarr")

        if not os.path.exists(file_path):
            print(
                f"Warning: Missing MBM file for {glacier_name} ({year}). Skipping..."
            )
            pred_gl.append(np.nan)
            continue

        ds = xr.open_dataset(file_path)
        pred_gl.append(ds.pred_masked.mean().item())

    # Create DataFrame
    glwide_MB = pd.DataFrame(pred_gl, index=years, columns=["MBM Balance"])

    # Merge with GLAMOS data
    glwide_MB = glwide_MB.join(GLAMOS_glwmb)

    # Drop NaN values to avoid plotting errors
    glwide_MB = glwide_MB.dropna()

    # Ensure data exists before plotting
    if glwide_MB.empty:
        print(f"Skipping {glacier_name}: No valid data to plot.")
        continue

    # Plot the data
    fig, ax = plt.subplots(figsize=(12, 6))
    glwide_MB.plot(ax=ax, marker="o", color=[color_xgb, color_tim])
    ax.set_title(f"{glacier_name.capitalize()} Glacier", fontsize=24)
    ax.set_ylabel("Mass Balance [m w.e.]", fontsize=18)
    ax.set_xlabel("Year", fontsize=18)
    ax.grid(True, linestyle="--", linewidth=0.5)

    # increase font size of legends
    ax.legend(fontsize=14)

    # Save the figure
    output_path = os.path.join(output_folder, f"{glacier_name}.png")
    plt.savefig(output_path, dpi=300, bbox_inches="tight")

    # Clear the plot to free memory
    plt.close(fig)

print("Processing complete. Figures saved in:", output_folder)

## Examine diff geod:

In [ ]:
# Define glacier name
glacier_name = "aletsch"
# Define the path to model predictions
path_results = os.path.join(PATH_PREDICTIONS, glacier_name)

# Load GLAMOS data
GLAMOS_glwmb = get_GLAMOS_glwmb(glacier_name, cfg)

# Extract available years from NetCDF filenames
years = sorted([
    int(f.split("_")[1]) for f in os.listdir(path_results)
    if f.endswith("_annual.zarr")
])

print(years)

# Extract model-predicted mass balance
pred_gl = []
for year in years:
    file_path = os.path.join(path_results,
                             f"{glacier_name}_{year}_annual.zarr")
    if not os.path.isdir(file_path):
        print(
            f"Warning: Missing MBM file for {glacier_name} ({year}). Skipping..."
        )
        pred_gl.append(np.nan)
        continue

    ds = xr.open_dataset(file_path)
    pred_gl.append(ds.pred_masked.mean().item())

# Create DataFrame
MBM_glwmb = pd.DataFrame(pred_gl, index=years, columns=["MBM Balance"])

# Merge with GLAMOS data
MBM_glwmb = MBM_glwmb.join(GLAMOS_glwmb)

# Drop NaN values to avoid plotting errors
MBM_glwmb = MBM_glwmb.dropna()

geoMB_target = geoMB_per_glacier.get(glacier_name, [])
geoMB_periods = periods_per_glacier.get(glacier_name, [])

df = pd.DataFrame(data={'geoMB_periods': geoMB_periods})
df['mbm_geod'], df['glamos_geod'], df['target_geod'] = np.nan, np.nan, np.nan

for i, period in enumerate(geoMB_periods):
    mbm_mb, glamos_mb = [], []
    for year in range(period[0], period[1] + 1):
        file_path = os.path.join(path_results,
                                 f"{glacier_name}_{year}_annual.zarr")
        ds = xr.open_dataset(file_path)
        mbm_mb.append(ds["pred_masked"].mean().values)

        file_ann = f"{year}_ann_fix_lv95.grid"
        grid_path_ann = os.path.join(cfg.dataPath, path_distributed_MB_glamos, 'GLAMOS',
                                     glacier_name, file_ann)

        metadata_ann, grid_data_ann = load_grid_file(grid_path_ann)
        ds_glamos_ann = convert_to_xarray_geodata(grid_data_ann, metadata_ann)
        ds_glamos_wgs84_ann = transform_xarray_coords_lv95_to_wgs84(
            ds_glamos_ann)
        glamos_mb.append(ds_glamos_wgs84_ann.mean().values)

    df['mbm_geod'].iloc[i] = np.nanmean(mbm_mb)
    df['glamos_geod'].iloc[i] = np.nanmean(glamos_mb)
    df['target_geod'].iloc[i] = geoMB_target[geoMB_periods.index(period)]

plot_mass_balance_comparison(glacier_name,
                             geoMB_periods,
                             MBM_glwmb,
                             df,
                             color_mbm=color_xgb,
                             color_model2=color_tim)


## Distributed GLAMOS:

In [ ]:
# Load stake data ONCE instead of for every glacier
stake_file = os.path.join(cfg.dataPath, path_PMB_GLAMOS_csv, "CH_wgms_dataset_all.csv")
df_stakes = pd.read_csv(stake_file)

### Validate against stake data:

In [ ]:
glacier_name = "aletsch"
year = 2022
fig = plot_mass_balance(glacier_name, year, df_stakes,
                        os.path.join(cfg.dataPath, path_distributed_MB_glamos, 'GLAMOS'),
                        PATH_PREDICTIONS)
plt.show()

In [ ]:
# Define glacier name
glacier_name = "aletsch"

# Get available years from filenames
years = sorted(
    map(int, [
        f.split("_")[1]
        for f in os.listdir(os.path.join(PATH_PREDICTIONS, glacier_name))
        if "_" in f and f.endswith("_annual.zarr")
    ]))

# if not exist create path:
if not os.path.exists(f"figures/dst_mb/{glacier_name}"):
    os.makedirs(f"figures/dst_mb/{glacier_name}")
# else empty:
else:
    emptyfolder(f"figures/dst_mb/{glacier_name}")

# Iterate through each year
for year in tqdm(years):
    print(f"Processing: {glacier_name}, Year: {year}")

    fig = plot_mass_balance(glacier_name, year, df_stakes,
                            os.path.join(cfg.dataPath, path_distributed_MB_glamos, 'GLAMOS'),
                            PATH_PREDICTIONS)

    # save figure
    output_path = os.path.join(f"figures/dst_mb/{glacier_name}",
                               f"{glacier_name}_{year}.png")
    # if fig not none:
    if fig:
        fig.savefig(output_path, dpi=300, bbox_inches="tight")
        plt.close()

## Input grids:

In [ ]:
glacier_name = 'aletsch'
year = 2010
month = 'jul'

# Feature columns:
feature_columns = [
    'ELEVATION_DIFFERENCE'
] + list(vois_climate) + list(vois_topographical) + ['pcsr']
all_columns = feature_columns + cfg.fieldsNotFeatures

glacier_file = os.path.join(cfg.dataPath, path_glacier_grid_glamos, glacier_name,
                            f"{glacier_name}_grid_{year}.parquet")

df_grid_monthly = pd.read_parquet(glacier_file)

# Correct climate grids:
# Take the biggest grid cell value for each month
for voi in [
        't2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10',
        'ALTITUDE_CLIMATE'
]:
    df_grid_monthly = correct_for_biggest_grid(
        df_grid_monthly, group_columns=["YEAR", "MONTHS"], value_column=voi)

# New elevation difference:
df_grid_monthly['ELEVATION_DIFFERENCE'] = df_grid_monthly[
    "POINT_ELEVATION"] - df_grid_monthly["ALTITUDE_CLIMATE"]

# apply T & P correction
# Apply temperature gradient correction
temp_grad = -6.5 / 1000
dpdz = 1.5 / 10000
c_prec = 1.434,
t_off = 0.617

# Apply temperature correction factor
df_grid_monthly['t2m_corr'] = df_grid_monthly['t2m'] + (
    df_grid_monthly['ELEVATION_DIFFERENCE'] * temp_grad)
df_grid_monthly['t2m_corr'] += t_off

# Apply elevation correction factor
df_grid_monthly['tp_corr'] = df_grid_monthly['tp'] * c_prec
df_grid_monthly['tp_corr'] += df_grid_monthly['tp_corr'] * (
    df_grid_monthly['ELEVATION_DIFFERENCE'] * dpdz)

# Rename aspect and slope to sgi
df_grid_monthly.rename(columns={
    'aspect': 'aspect_sgi',
    'slope': 'slope_sgi'
},
                       inplace=True)
df_grid_monthly['POINT_ELEVATION'] = df_grid_monthly['topo']
df_grid_monthly.drop_duplicates(inplace=True)  # remove duplicates
df_grid_monthly = df_grid_monthly[all_columns]

df_grid_month = df_grid_monthly[df_grid_monthly.MONTHS == month]

In [ ]:
# Define features to plot
v_plot = ["tp_corr", "ELEVATION_DIFFERENCE"
          ] + vois_topographical + ['slhf', 'sshf']

# Ensure the grid layout fits the number of plots dynamically
n_rows = 3
n_cols = min(3, len(v_plot))  # Ensure no more than 4 columns

# Create figure and axes
fig, axs = plt.subplots(n_rows, n_cols, figsize=(10, 10))

# Flatten the axes array for easier indexing
axs = axs.flatten()

# Plot each feature
for i, voi in enumerate(v_plot):
    sns.scatterplot(data=df_grid_month,
                    x="POINT_LON",
                    y="POINT_LAT",
                    hue=voi,
                    ax=axs[i],
                    palette="twilight_shifted",
                    s=2)
    axs[i].set_title(voi)
    axs[i].set_ylabel("Lat")
    axs[i].set_xlabel("Lon")
    axs[i].legend().remove()
    axs[i].grid(False)

# Remove unused subplots if any
for j in range(i + 1, len(axs)):
    fig.delaxes(axs[j])

# Adjust layout for clarity
plt.tight_layout()
plt.show()

## Stake coverage:

In [ ]:
# Define year and month for analysis
year = 2018
month = "jul"

# Ensure the output folder is empty
output_folder = "figures/stake_coverage/"
emptyfolder(output_folder)

# Load stake data ONCE instead of for every glacier
stake_file = os.path.join(cfg.dataPath, path_PMB_GLAMOS_csv, "CH_wgms_dataset_all.csv")
if not os.path.exists(stake_file):
    print("Error: Stake data file not found. Exiting.")
else:
    df_stakes = pd.read_csv(stake_file)

# Iterate through each glacier
for glacier_name in tqdm(glacier_list, desc="Processing glaciers"):
    # Construct glacier grid file path
    glacier_file = os.path.join(cfg.dataPath, path_glacier_grid_glamos, glacier_name,
                                f"{glacier_name}_grid_{year}.parquet")

    if not os.path.exists(glacier_file):
        print(f"Skipping {glacier_name}: Grid file not found.")
        print(glacier_file)
        continue  # Skip if glacier grid file is missing

    # Load and preprocess glacier grid data
    df_grid_monthly = pd.read_parquet(glacier_file)
    df_grid_monthly = correct_vars_grid(df_grid_monthly)

    # Rename aspect and slope to sgi
    df_grid_monthly.rename(columns={
        "aspect": "aspect_sgi",
        "slope": "slope_sgi"
    },
                           inplace=True)
    df_grid_monthly["POINT_ELEVATION"] = df_grid_monthly["topo"]
    df_grid_monthly.drop_duplicates(inplace=True)  # Remove duplicates

    # Filter dataset for selected month
    df_grid_month = df_grid_monthly[df_grid_monthly.MONTHS == month]

    # Filter stakes for this glacier
    df_stakes_glacier = df_stakes[df_stakes["GLACIER"] == glacier_name].copy()

    # If no stake data available for this glacier, continue
    if df_stakes_glacier.empty:
        print(f"Skipping {glacier_name}: No stake data available.")
        continue

    # Convert to GeoDataFrame for geospatial visualization
    df_stakes_glacier["geometry"] = gpd.points_from_xy(
        df_stakes_glacier.POINT_LON, df_stakes_glacier.POINT_LAT)
    gdf_stakes = gpd.GeoDataFrame(df_stakes_glacier, crs="EPSG:4326")

    # Load SGI grid
    sgi_file = os.path.join(cfg.dataPath, path_SGI_topo,
                            f"xr_masked_grids/{glacier_name}.zarr")

    if not os.path.exists(sgi_file):
        print(f"Skipping {glacier_name}: SGI NetCDF file not found.")
        continue

    sgi_grid = xr.open_dataset(sgi_file)

    # Create figure and plot
    fig, ax = plt.subplots(figsize=(5, 5))

    # Plot glacier mask
    sgi_grid.glacier_mask.plot(cmap="binary", ax=ax, alpha=0.5)

    # Scatter plot of the monthly glacier grid with elevation difference
    scatter = sns.scatterplot(data=df_grid_month,
                              x="POINT_LON",
                              y="POINT_LAT",
                              hue="ELEVATION_DIFFERENCE",
                              ax=ax,
                              s=5,
                              palette="twilight_shifted",
                              legend=True)

    # Plot stake locations
    gdf_stakes.plot(ax=ax,
                    color=color_tim,
                    edgecolor="black",
                    markersize=20,
                    label="Stakes")

    # Remove automatic Seaborn legend
    ax.legend().remove()

    # Set title
    ax.set_title(f"{glacier_name}")

    # Save figure
    output_path = os.path.join(output_folder, f"{glacier_name}.png")
    plt.savefig(output_path, dpi=300, bbox_inches="tight")

    # Clear the plot to free memory
    plt.close(fig)

print("Processing complete. Figures saved to:", output_folder)

## S2 Sentinel data:

### Get S2 rasters:
Get the names of all satellite nc files and classifiy them per month and hydrological year.

In [ ]:
# Sentinel-2
src_crs = 'EPSG:4326'  # Original CRS (lat/lon) wgs84

# Organize the rasters by hydrological year
satellite_years = [2015, 2016, 2017, 2018, 2019, 2020, 2021]

rasters_S2 = organize_rasters_by_hydro_year(cfg.dataPath+path_S2, satellite_years)

# Print the organized rasters
for hydro_year, months in rasters_S2.items():
    print(f"-----------------\nHydrological Year: {hydro_year}")
    for month, files in months.items():
        print(f"  {month}: {files}")

### Create gdf for S2 rasters:
Create panadas geodataframe for each satellite raster.

In [ ]:
satellite_glaciers = [
    'adler', 'aletsch', 'allalin', 'basodino', 'claridenL', 'claridenU',
    'findelen', 'gries', 'hohlaub', 'limmern', 'oberaar', 'plattalva', 'rhone',
    'sanktanna', 'schwarzbach', 'schwarzberg'
]

processed_gl = os.listdir(PATH_PREDICTIONS)

# list of gl in processed_gl that are also in satellite_glaciers
satellite_glaciers = [gl for gl in processed_gl if gl in satellite_glaciers]
satellite_glaciers

## SGI rasters:

### S2 rasters over glaciers:

In [ ]:
RUN = True
if RUN:
    # emptyfolder(os.path.join(cfg.dataPath, path_S2, 'perglacier/'))
    # For each year and month where there is S2 data
    for hydro_year, months in tqdm(rasters_S2.items(),
                                   position=0,
                                   desc='Hydrological Years'):
        for month, files in tqdm(months.items(),
                                 position=1,
                                 leave=False,
                                 desc='Months'):

            # More than one if S2 A and B flew in the same month
            for raster_name in files:
                # Create raster for that year & month:
                file_date = datetime.strptime(
                    raster_name.split('_')[3][:8],
                    "%Y%m%d")  # Extract the 8-digit date (YYYYMMDD)

                # Path to the raster for that year & month:
                raster_path = os.path.join(cfg.dataPath, path_S2, file_date.strftime('%Y'),
                                           raster_name)

                # Creates a geodataframe from the .tif raster
                # This is the longest part of the code
                geo_data_S2 = mbm.geodata.GeoData(pd.DataFrame)
                gdf_S2 = geo_data_S2.raster_to_gpd(raster_path)
                geo_data_S2.set_gdf(gdf_S2)

                # For each glacier, clip the S2 raster to the glacier extent
                for glacierName in tqdm(satellite_glaciers,
                                        position=2,
                                        leave=False,
                                        desc='Glaciers'):

                    # Load MB predictions for that year and month
                    path_nc_wgs84 = PATH_PREDICTIONS + f"{glacierName}/"
                    filename_nc = f"{glacierName}_{hydro_year}_{cfg.month_abbr_hydr[month]}.nc"

                    # check if file exists:
                    if not os.path.exists(
                            os.path.join(path_nc_wgs84, filename_nc)):
                        continue

                    # Open xarray dataset and set to class:
                    geoData_gl = mbm.geodata.GeoData(pd.DataFrame)
                    geoData_gl.set_ds_latlon(filename_nc, path_nc_wgs84)

                    # Clip the S2 dataframe to the glacier extent
                    # and resample it to the glacier resolution
                    gdf_raster_res = mbm.geodata.GeoData.resample_satellite_to_glacier(
                        geoData_gl.gdf, gdf_S2)

                    # In case the glacier is outside of the bounds of the raster
                    if gdf_raster_res is 0:
                        continue
                    # In case the raster is empty where the glacier is
                    elif gdf_raster_res is 1:
                        continue
                    else:
                        pass

                    # Check the percentage of cloud cover
                    cloud_cover_glacier = gdf_raster_res.classes[
                        gdf_raster_res.classes ==
                        5].count() / gdf_raster_res.classes.count()

                    # If the cloud cover is too high, skip the glacier
                    if cloud_cover_glacier > 0.5:
                        continue

                    # Save the glacier raster:
                    S2_gl_name = '{}_{}.geojson'.format(
                        glacierName, file_date.strftime('%Y_%m_%d'))
                    S2_gl_path = os.path.join(cfg.dataPath, path_S2, 'perglacier',
                                              S2_gl_name)
                    gdf_raster_res.to_file(S2_gl_path, driver="GeoJSON")

## Snow cover:

In [ ]:
RUN = True
if RUN:

    # first delete results file
    if os.path.exists(f'results/snow_cover.csv'):
        os.remove(f'results/snow_cover.csv')

    # Add header
    with open(f'results/snow_cover.csv', 'w') as f:
        f.write(
            "year,month,monthNb,raster_date,glacier_name,snow_cover_S2,snow_cover_glacier\n"
        )

    # All rasters resampled over glaciers:
    rasters_resampled = os.listdir(os.path.join(cfg.dataPath, path_S2, 'perglacier'))

    for raster_res in tqdm(rasters_resampled):
        # Extract glacier name
        glacierName = raster_res.split('_')[0]

        # Extract date from satellite raster
        match = re.search(r"(\d{4})_(\d{2})_(\d{2})", raster_res)
        year, month, day = match.groups()
        date_str = match.group(1) + '-' + match.group(2) + '-' + match.group(3)
        raster_date = datetime.strptime(date_str, "%Y-%m-%d")

        # Find closest hydrological year and month
        closest_month, hydro_year = get_hydro_year_and_month(raster_date)
        monthNb = month_abbr_hydr_full[closest_month]

        if hydro_year > 2021:
            continue

        # Read satellite raster over glacier (previously resampled)
        gdf_S2_res = gpd.read_file(
            os.path.join(cfg.dataPath, path_S2, 'perglacier', raster_res))

        # Calculate percentage of snow cover (class 1)
        snow_cover_S2 = IceSnowCover(gdf_S2_res, gdf_S2_res)

        # Load MB predictions for that year and month
        path_nc_wgs84 = PATH_PREDICTIONS + f"{glacierName}/"
        filename_nc = f"{glacierName}_{hydro_year}_{monthNb}.nc"

        geoData_gl = mbm.geodata.GeoData(pd.DataFrame)
        geoData_gl.set_ds_latlon(filename_nc, path_nc_wgs84)
        geoData_gl.classify_snow_cover(tol=0.1)

        snow_cover_glacier = IceSnowCover(geoData_gl.gdf, gdf_S2_res)

        # Save the results
        with open(f'results/snow_cover.csv', 'a') as f:
            f.write(
                f"{hydro_year},{closest_month},{monthNb},{date_str},{glacierName},{snow_cover_S2},{snow_cover_glacier}\n"
            )

In [ ]:
df = pd.read_csv(f'results/snow_cover.csv').sort_values(by=['year', 'month'],
                                                        ascending=True)
# remove october and september
df = df[~df.month.isin(['oct', 'sep'])]
fig, axs = plot_snow_cover_scatter(df, add_corr=False)
plt.show()

In [ ]:
df_gl = df[(df.glacier_name == 'aletsch') & ~df.month.isin(['oct', 'sep'])]
fig, axs = plot_snow_cover_scatter_combined(df_gl)

df_gl = df[(df.glacier_name == 'gries') & ~df.month.isin(['oct', 'sep'])]
fig, axs = plot_snow_cover_scatter_combined(df_gl)

df_gl = df[(df.glacier_name == 'rhone') & ~df.month.isin(['oct', 'sep'])]
fig, axs = plot_snow_cover_scatter_combined(df_gl)

df_gl = df[(df.glacier_name == 'adler') & ~df.month.isin(['oct', 'sep'])]
fig, axs = plot_snow_cover_scatter_combined(df_gl)

plt.show()

In [ ]:
# Loop through the raster files and plot geoplots
gl = 'aletsch'
yr = 2021
rasters_gl = [
    f for f in os.listdir(os.path.join(cfg.dataPath, path_S2, 'perglacier'))
    if (gl in f) and (str(yr) in f)
]
rasters_gl.sort()
for raster_res in rasters_gl:
    plot_snow_cover_geoplots(raster_res, cfg.dataPath+path_S2, month_abbr_hydr_full)


## Snowline:

In [ ]:
# Loop through the raster files and plot geoplots
gl = 'aletsch'
yr = 2021
rasters_gl = [
    f for f in os.listdir(os.path.join(cfg.dataPath, path_S2, 'perglacier'))
    if (gl in f) and (str(yr) in f)
]
rasters_gl.sort()
for raster_res in rasters_gl:
    plot_snow_cover_geoplots(raster_res,
                             cfg.dataPath+path_S2,
                             month_abbr_hydr_full_hydr,
                             add_snowline=True,
                             band_size=50,
                             percentage_threshold=50)


In [ ]:
# Loop through the raster files and plot geoplots
gl = 'rhone'
yr = 2021
rasters_gl = [
    f for f in os.listdir(os.path.join(cfg.dataPath, path_S2, 'perglacier'))
    if (gl in f) and (str(yr) in f)
]
rasters_gl.sort()
for raster_res in rasters_gl:
    plot_snow_cover_geoplots(raster_res,
                             cfg.dataPath+path_S2,
                             month_abbr_hydr_full_hydr,
                             add_snowline=True,
                             band_size=50,
                             percentage_threshold=50)


## Missed glaciers:

#### Problem 1: Glacier is in regions where raster is NaN

In [ ]:
year = 2015
glacierName = 'taelliboden'
filename_nc = f"{glacierName}_{year}.nc"
path_nc_wgs84 = f"results/nc/var_normal/{glacierName}/wgs84/"
path_nc_wgs84_corr = f"results/nc/var_corr/{glacierName}/wgs84/"

gdf_raster = createRaster(raster_a_2015)

# Calculate snow cover for glacier
gdf_glacier, snow_cover_glacier, ice_cover_glacier = snowCover(
    path_nc_wgs84, filename_nc)
# Corrected for temperature & precipitation
gdf_glacier, snow_cover_glacier_corr, ice_cover_glacier_corr = snowCover(
    path_nc_wgs84_corr, filename_nc)

# Clip the raster to the glacier extent and resample it to the glacier resolution
# gdf_raster_res = resampleRaster(gdf_glacier, gdf_raster)

bounding_box = gdf_glacier.total_bounds  # [minx, miny, maxx, maxy]
raster_bounds = gdf_raster.total_bounds  # [minx, miny, maxx, maxy]

# Check if glacier bounds are within raster bounds
if not (bounding_box[0] >= raster_bounds[0]
        and  # minx of glacier >= minx of raster
        bounding_box[1] >= raster_bounds[1]
        and  # miny of glacier >= miny of raster
        bounding_box[2] <= raster_bounds[2]
        and  # maxx of glacier <= maxx of raster
        bounding_box[3]
        <= raster_bounds[3]  # maxy of glacier <= maxy of raster
        ):
    print(f"Glacier {glacierName} is out of bounds")

bbox_polygon = box(*bounding_box)

# The raster might have no data (NaN values) in the region of the glacier:
bounding_box = [7.8, 45.95854232, 8, 46.1]
bbox_polygon = box(*bounding_box)
gfd_res = gdf_raster[gdf_raster.intersects(bbox_polygon)]
ax = gfd_res.plot(color='blue', alpha=0.5)
gdf_glacier.plot(ax=ax, color='red')

## Transform to tif rasters for QGIS:

### One year:

In [ ]:
glacierName = 'aletsch'
year = 2021

filename_nc = f"{glacierName}_{year}.nc"
path_nc_lv95 = f"results/nc/{glacierName}/lv95/"
path_nc_wgs84 = f"results/nc/{glacierName}/wgs84/"

filename_tif = f"{glacierName}_{year}.tif"
path_tif_wgs84 = f"results/tif/{glacierName}/wgs84/"
path_tif_lv95 = f"results/tif/{glacierName}/lv95/"

createPath(path_tif_lv95)
createPath(path_tif_wgs84)

In [ ]:
gdf, gdf_class, raster_data, extent = TransformToRaster(
    filename_nc, filename_tif, path_nc_wgs84, path_tif_wgs84, path_tif_lv95)

In [ ]:
API_KEY = "000378bd-b0f0-46e2-a46d-f2165b0c6c02"
provider = cx.providers.Stadia.StamenTerrain(api_key=API_KEY)
provider["url"] = provider["url"] + f"?api_key={API_KEY}"

vmin, vmax = gdf.data.min(), gdf.data.max()
norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
gdf_clean = gdf.dropna(subset=["data"])
gdf_clean.plot(
    column="data",  # Column to visualize
    cmap="RdBu",  # Color map suitable for glacier data
    norm=norm,
    legend=True,  # Display a legend
    ax=axs[0],
    markersize=5,  # Adjust size if points are too small or large
    missing_kwds={"color": "lightgrey"}  # Define color for NaN values
)
cx.add_basemap(axs[0], crs=gdf.crs, source=provider)

gdf_clean = gdf_class.dropna(subset=["data"])
gdf_clean.plot(
    column="data",  # Column to visualize
    legend=True,  # Display a legend
    ax=axs[1],
    markersize=5,  # Adjust size if points are too small or large
    missing_kwds={"color": "lightgrey"}  # Define color for NaN values
)
cx.add_basemap(axs[1], crs=gdf.crs, source=provider)
plt.tight_layout()

In [ ]:
vmin, vmax = gdf.data.min(), gdf.data.max()

# Calculate the relative position of 0
relative_position = (0 - vmin) / (vmax - vmin) * 100

# Print the result
print(f"The relative position of 0 is {relative_position:.2f}%")

### All years:

In [ ]:
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021]
for year in years:
    print(year)
    for month in month_abbr_hydr_full_hydr:
        monthNb = month_abbr_hydr_full_hydr[month]

        filename_nc = f"{glacierName}_{year}_{monthNb}.nc"
        path_nc_lv95 = f"results/nc/{glacierName}/lv95/"
        path_nc_wgs84 = f"results/nc/{glacierName}/wgs84/"

        filename_tif = f"{glacierName}_{year}_{monthNb}.tif"
        path_tif_wgs84 = f"results/tif/{glacierName}/wgs84/"
        path_tif_lv95 = f"results/tif/{glacierName}/lv95/"  # normally EPSG Code: 2056

        gdf, gdf_class, raster_data, extent = TransformToRaster(
            filename_nc, filename_tif, path_nc_wgs84, path_tif_wgs84,
            path_tif_lv95)

In [ ]:
API_KEY = "000378bd-b0f0-46e2-a46d-f2165b0c6c02"
provider = cx.providers.Stadia.StamenTerrain(api_key=API_KEY)
provider["url"] = provider["url"] + f"?api_key={API_KEY}"

vmin, vmax = gdf.data.min(), gdf.data.max()
norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
fig, axs = plt.subplots(1, 2, figsize=(5, 5))
gdf_clean = gdf.dropna(subset=["data"])
gdf_clean.plot(
    column="data",  # Column to visualize
    cmap="RdBu",  # Color map suitable for glacier data
    norm=norm,
    legend=True,  # Display a legend
    ax=axs[0],
    markersize=5,  # Adjust size if points are too small or large
    missing_kwds={"color": "lightgrey"}  # Define color for NaN values
)
cx.add_basemap(axs[0], crs=gdf.crs, source=provider)

gdf_clean = gdf_class.dropna(subset=["data"])
gdf_clean.plot(
    column="data",  # Column to visualize
    legend=True,  # Display a legend
    ax=axs[1],
    markersize=5,  # Adjust size if points are too small or large
    missing_kwds={"color": "lightgrey"}  # Define color for NaN values
)
cx.add_basemap(axs[1], crs=gdf.crs, source=provider)

### Step by step example of one file:

### Step 1: Open the NetCDF file


In [ ]:
ds_xy = xr.open_dataset(path_nc_lv95 + filename_nc)
ds_latlon = xr.open_dataset(path_nc_wgs84 + filename_nc)

# Smoothing
ds_xy_g = GaussianFilter(ds_xy)
ds_latlon_g = GaussianFilter(ds_latlon)

# Show effet of Smoothing:
vmin, vmax = np.min([
    ds_xy.pred_masked.min().values,
    ds_xy_g.pred_masked.min()
]), np.max([ds_xy.pred_masked.max().values,
            ds_xy_g.pred_masked.max()])
max_abs_value = max(abs(vmin), abs(vmax))
norm = mcolors.TwoSlopeNorm(vmin=-max_abs_value, vcenter=0, vmax=max_abs_value)

fig, axs = plt.subplots(1, 2, figsize=(20, 10))
ds_xy.pred_masked.plot.imshow(cmap='RdBu', norm=norm, ax=axs[0])
axs[0].set_title('Original')

# Plot or analyze `smoothed_data` as needed
ds_xy_g.pred_masked.plot.imshow(cmap='RdBu', norm=norm, ax=axs[1])
axs[1].set_title('Gaussian Filter')

# print min and max values
print(ds_xy.pred_masked.min().values, ds_xy.pred_masked.max().values)
print(ds_xy_g.pred_masked.min().values, ds_xy_g.pred_masked.max().values)

### Step 2: transform to geopandas

In [ ]:
gdf, lon, lat = toGeoPandas(ds_latlon_g)

# Reproject to LV95 (EPSG:2056) swiss coordinates
# gdf_lv95 = gdf.to_crs("EPSG:2056")

API_KEY = "000378bd-b0f0-46e2-a46d-f2165b0c6c02"
provider = cx.providers.Stadia.StamenTerrain(api_key=API_KEY)
provider["url"] = provider["url"] + f"?api_key={API_KEY}"

fig, ax = plt.subplots(figsize=(5, 5))
gdf_clean = gdf.dropna(subset=["data"])
gdf_clean.plot(
    column="data",  # Column to visualize
    cmap="Reds",  # Color map suitable for glacier data
    legend=True,  # Display a legend
    ax=ax,
    markersize=5,  # Adjust size if points are too small or large
    missing_kwds={"color": "lightgrey"}  # Define color for NaN values
)
cx.add_basemap(ax, crs=gdf.crs, source=provider)

### Step 3: transform to raster

In [ ]:
# Transform to raster from geopandas
raster_data, extent = toRaster(gdf,
                               lon,
                               lat,
                               file_name=path_tif_wgs84 + filename_tif)

# reproject raster to Swiss coordinates (LV95)
reproject_raster_to_lv95(path_tif_wgs84 + filename_tif,
                         path_tif_lv95 + filename_tif)

### Opt step 4: for clariden
Need to merge two rasters

In [ ]:
if 'clariden' in glacierName:
    merge_rasters('results/tif/claridenL_2022_w_lv95.tif',
                  'results/tif/claridenU_2022_w_lv95.tif',
                  'results/tif/clariden_2022_w_lv95.tif')